# **The Real Problem**

The goal of the Fake News Challenge is to explore how artificial intelligence technologies, particularly machine learning and natural language processing, might be leveraged to combat the fake news problem. We believe that these AI technologies hold promise for significantly automating parts of the procedure human fact checkers use today to determine if a story is real or a hoax. h
 
Assessing the veracity of a news story is a complex and cumbersome task, even for trained experts. Fortunately, the process can be broken down into steps or stages. A helpful first step towards identifying fake news is to understand what other news organizations are saying about the topic. We believe automating this process, called Stance Detection, could serve as a useful building block in an AI-assisted fact-checking pipeline. 
 
Stance Detection involves estimating the relative perspective (or stance) of two pieces of text relative to a topic, claim or issue. This extends the work of Ferreira & Vlachos. The task is to estimate the stance of a body text from a news article relative to a headline. Specifically, the body text may agree, disagree, discuss or be unrelated to the headline. 
 
# **Project Description **

Given the huge impact of social networks, online content plays an important role in forming or changing the opinions of people. Unlike traditional journalism where only certain news organizations can publish content, online journalism has given chance even for individuals to publish. This has its own advantages like individual empowerment but has given a chance to a lot of malicious entities to spread misinformation for their own benefit. As reported by many organizations in recent history, this even has an influence on major events like the outcome of elections. Therefore, it is of great importance now, to have some sort of automated classification of news stories. 	 	 	 	 
 
In this work, we will implement different deep neural architectures using word embeddings for identifying the stance detection of different news stories. 

# **Dataset**

The dataset is taken from Fake News Challenge, where the goal of the challenge is not to directly identify whether a headline or article is “fake” or not, which is arguably a highly subjective question, and one that even skilled humans may have difficulty answering. 
 
The challenge is organized around the more well-defined problem of “stance detection,” which involves comparing a headline with a body of text from a news article to determine what relationship (if any) exists between the two.  
 
There are 4 possible classifications:  
 
1.	The article text agrees with the headline.  
2.	The article text disagrees with the headline.  
3.	The article text is a discussion of the headline, without taking a position on it.	 
4.	The article text is unrelated to the headline (i.e. it doesn’t address the same topic).  
 
 
Presumably, a classifier that can solve the stance detection problem with high accuracy might effectively be used either as a tool for humans working to identify fake news (e.g., retrieving articles that agree, disagree and discuss a headline), or as a building block for a more elaborate future AI system that would try to identify the actual truthfulness of news stories (e.g., using credible sources to classify).



# **I will follow these steps: **

Step 1: Understanding how to convert words into vectors and make them ready for the model. 
This is an important step and will be the same for almost any problem in NLP when dealing with words. 
 
Step 2: Understanding the working of Sequential Models like RNN, LSTM.. and practically building them in Keras and build the model ready. 
 
Step 3: Train the model and report the accuracy score. 
 
Step 4: Add checkpoints as callbacks to the model and save the model for each epoch. This callback saves the model after each epoch, which can be handy for when you are running long-term training. 
By loading the saved weight files we can start training the model which starts the learning from there. 

Step 5 (will try if time permits): Add attention to the above model and report the change in accuracy. And explain how and why attention has improved (or disturbed) the model. 


# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

**Mount the google drive to access required project files**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Path for Project files on google drive


In [0]:
project_path = "/content/drive/My Drive/Colab Notebooks/Seq Models in NLP/Project 2/"

### Loading the Glove Embeddings

I have commented this code as the files have been unzipped once and not required with every code run

In [0]:

# from zipfile import ZipFile
# with ZipFile(project_path+'glove.6B.zip', 'r') as z:
#   z.extractall()

# Load the dataset 

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import numpy as np
import pandas as pd

In [0]:
df_bodies = pd.read_csv(project_path+'train_bodies.csv')


In [8]:
df_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [0]:
df_stances = pd.read_csv(project_path+'train_stances.csv')

In [10]:
df_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
df = df_bodies.merge(df_stances, left_on='Body ID', right_on='Body ID')


<h2> Run `dataset.head() and Check the output </h3>

In [12]:
df.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### lets set the following required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

reference: read  [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. 

#### Import the Tokenizer from keras preprocessing text

In [0]:
%tensorflow_version 1.x

In [0]:
import tensorflow as tf

In [18]:
%tensorflow_version

Currently selected TF version: 1.x
Available versions:
* 1.x
* 2.x


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS) # num_words -> Vocablury size

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
t.fit_on_texts(df['articleBody']) 
t.fit_on_texts(df['Headline'])

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [21]:

len(t.word_counts)

27873

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize 

### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = [] 
articles = [] 

In [0]:
texts = df['articleBody']

In [0]:
for text in texts:
    articles.append(sent_tokenize(text))


## Check 2:

first element of texts and articles should be as given below. 

In [26]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [27]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

In [28]:
articles

[["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
  "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
  'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
  'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
  'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
  'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
  'He said it is still not clear if the meteorite disintegrated or

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [29]:
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2



In [0]:
data =  np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH), dtype = int)

In [32]:
print (len(articles))

49972


In [33]:
# tokenizing the text 

for index_one, article in enumerate(articles):
  article = article[0:MAX_SENTS]
  for index_two, sentence in enumerate(article):
    list_of_words = text_to_word_sequence(sentence)[0:MAX_SENT_LENGTH]
    encodings = []
    print(index_one, index_two)
    for word in list_of_words:
      encoding = t.word_index.get(word)
      encodings.append(encoding)
    if(len(encodings) < MAX_SENT_LENGTH):
      encodings.extend([0] * (MAX_SENT_LENGTH - len(encodings)))
    data[index_one][index_two] = encodings

Streaming output truncated to the last 5000 lines.
49699 2
49699 3
49699 4
49699 5
49699 6
49699 7
49699 8
49699 9
49699 10
49699 11
49699 12
49699 13
49699 14
49700 0
49700 1
49700 2
49700 3
49700 4
49700 5
49700 6
49700 7
49700 8
49700 9
49700 10
49700 11
49700 12
49700 13
49700 14
49701 0
49701 1
49701 2
49701 3
49701 4
49701 5
49701 6
49701 7
49701 8
49701 9
49701 10
49701 11
49701 12
49701 13
49701 14
49702 0
49702 1
49702 2
49702 3
49702 4
49702 5
49702 6
49702 7
49702 8
49702 9
49702 10
49702 11
49702 12
49702 13
49702 14
49703 0
49703 1
49703 2
49703 3
49703 4
49703 5
49703 6
49703 7
49703 8
49703 9
49703 10
49703 11
49703 12
49703 13
49703 14
49704 0
49704 1
49704 2
49704 3
49704 4
49704 5
49704 6
49704 7
49704 8
49704 9
49704 10
49704 11
49704 12
49704 13
49704 14
49704 15
49704 16
49704 17
49704 18
49704 19
49705 0
49705 1
49705 2
49705 3
49705 4
49705 5
49705 6
49705 7
49705 8
49705 9
49705 10
49705 11
49705 12
49705 13
49705 14
49705 15
49705 16
49705 17
49705 18
49705 19


### Check 3:

Accessing first element in data should give something like given below.

In [34]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

## Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly.

In [0]:
texts_heading = df.Headline
articles_heading = []
for text_heading in texts_heading:
  article_heading = nltk.sent_tokenize(text_heading)
  articles_heading.append(article_heading)

In [36]:
texts_heading[0]
articles_heading[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [37]:
print(len(articles_heading))

49972


In [0]:
data_heading =  np.zeros((len(articles_heading), MAX_SENTS_HEADING, MAX_SENT_LENGTH), dtype = int)

In [0]:
for index_one, heading in enumerate(articles_heading):
  heading = heading[0:MAX_SENTS_HEADING]
  for index_two, head in enumerate(heading):
    list_of_words = text_to_word_sequence(head)[0:MAX_SENT_LENGTH]
    encodings = []
    for word in list_of_words:
      encoding = t.word_index.get(word)
      encodings.append(encoding)
    if(len(encodings) < MAX_SENT_LENGTH):
      encodings.extend([0] * (MAX_SENT_LENGTH - len(encodings)))
    data_heading[index_one][index_two] = encodings

In [40]:
data_heading[49971, :, :]

array([[6105, 6363, 2165, 2157, 1856, 1998,   14,  284,  145, 2379,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(df_stances.Stance)



In [42]:
labels.head()


,agree,disagree,discuss,unrelated
0,0,0,0,1
1,1,0,0,0
2,0,0,0,1
3,0,0,0,1
4,0,1,0,0


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [43]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
#labels = labels[indices]
labels = labels.iloc[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split


In [0]:

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [48]:
int(len(data) * 0.8)

39977

In [0]:
x_train = data[0 : int(len(data) * 0.8) + 1, :, :]
x_val = data[int(len(data) * 0.8) + 1 : len(data), :, :]

x_heading_train = data_heading[0 : int(len(data_heading) * 0.8) + 1, :, :]
x_heading_val = data_heading[int(len(data_heading) * 0.8) + 1 : len(data_heading), :, :]

y_train = labels.iloc[0:int(len(data) * 0.8) + 1, :]
y_val = labels.iloc[int(len(data) * 0.8) + 1 : len(data), :]

In [50]:
print(x_val.shape)

(9994, 20, 20)


### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [51]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


In [55]:
x_train.shape

(39978, 20, 20)

In [0]:
x_train1=x_train.swapaxes(0,1).reshape(39978, 400)

In [0]:
x_train = np.reshape(x_train, (39978, 400))
x_val = np.reshape(x_val, (9994, 400))

In [61]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 400)
(39978, 4)
(9994, 400)
(9994, 4)


### Create embedding matrix with the glove embeddings


create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [0]:
# #load the whole embedding into memory
# embeddings_index = dict()
# f = open('./glove.6B.100d.txt')
# for line in f:
# 	values = line.split()
# 	word = values[0]
# 	coefs = np.asarray(values[1:], dtype='float32')
# 	embeddings_index[word] = coefs
# f.close()
# print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
# embedding_matrix = np.zeros((vocab_size, 100))


# for word, i in t.word_index.items():
# 	embedding_vector = embeddings_index.get(word)
# 	if embedding_vector is not None:
# 		embedding_matrix[i] = embedding_vector

In [65]:
# load the whole embedding into memory
embeddings_index = dict()
f = open( project_path +  'glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((MAX_NB_WORDS, 100))


for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
      if(i >= MAX_NB_WORDS):
        continue
      else:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score.

### Import layers from Keras to build the model

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional, GlobalAveragePooling1D

tf.set_random_seed(100)

### Model

In [67]:
model = keras.Sequential()
model.add(keras.layers.Embedding(len(t.word_index) + 1, 100))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(64, activation='relu'))

# Dropout for regularization
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(4, activation="sigmoid"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2787400   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 6

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import Callback

checkpoint = ModelCheckpoint("init_model.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=1, mode='auto')

### Compile and fit the model

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [71]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs = 10, batch_size = 50, verbose = 1)## Train the model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 39978 samples, validate on 9994 samples
Epoch 1/10
39978/39978 [==============================] - 7s 165us/sample - loss: 0.8328 - acc: 0.7296 - val_loss: 0.7908 - val_acc: 0.7358
Epoch 2/10
39978/39978 [==============================] - 5s 118us/sample - loss: 0.8035 - acc: 0.7302 - val_loss: 0.7892 - val_acc: 0.7358
Epoch 3/10
39978/39978 [==============================] - 5s 116us/sample - loss: 0.8008 - acc: 0.7302 - val_loss: 0.7959 - val_acc: 0.7358
Epoch 4/10
39978/39978 [==============================] - 5s 113us/sample - loss: 0.7977 - acc: 0.7302 - val_loss: 0.7926 - val_acc: 0.7358
Epoch 5/10
39978/39978 [==============================] - 5s 119us/sample - loss: 0.7956 - acc: 0.7302 - val_loss: 0.7941 - val_acc: 0.7358
Epoch 6/10
39978/39978 [==============================] - 5s 117us/sample - loss: 0.7938 - acc: 0.7302 - val_loss: 0.7949 - val_acc: 0.7358
Epoch 7/10
39978/

In [0]:
# Get back the encoding values to compare with the predictions for getting accuracy score
i, val = np.where(y_val)

In [0]:
#Get predictions for validation data
pred = model.predict_classes(x_val)

In [74]:
from sklearn.metrics import accuracy_score

print ('Model Accuracy: ', accuracy_score(val, pred))

Model Accuracy:  0.7358415049029418
